In [1]:
print("hhi")

hhi


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()


True

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
google_gemini = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0.9)
google_gemini.invoke("What is the capital of France?").content


[{'type': 'text',
  'text': 'The capital of France is **Paris**.',
  'extras': {'signature': 'EnAKbgG+Pvb7MX2OvjlrQUoaNJ8isqpV+MkKAaOhVhGVlNoZpOygzdOevB/u6XdOiIZ0kWOH+DxqEviijoUe7yM89Y0IV1n81egxpQx+KAG2gsQeCn4wHnqmf11zI+R+1p7nT4zufjp6LZSB3QtwpaMF'}}]

In [11]:
from typing_extensions import Annotated,TypedDict
import operator

In [12]:
from langchain_core.messages import ChatMessage, HumanMessage, SystemMessage, AIMessage,AnyMessage

In [13]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    

In [14]:
def llm_call(state: GraphState) -> dict:
    """Call the LLM with messages and append AI response."""
    response = google_gemini.invoke(state["messages"])
    return {
        "messages": [AIMessage(content=response.content)]
    }


def token_counter(state: GraphState) -> dict:
    """Count tokens (simple word count) in the last AI message."""
    last_msg = state["messages"][-1]
    text = last_msg.content
    token_number = len(text.split())
    summary = f"Total token number in the generated answer (word count) is {token_number}"
    return {
        "messages": [AIMessage(content=summary)]
    }

In [15]:
from langgraph.graph import StateGraph


In [17]:
builder=StateGraph(GraphState)


In [18]:
builder.add_node(llm_call)
builder.add_node(token_counter)


In [ ]:
builder.set_entry_point("llm_call")
builder.add_node("llm_call", llm_call)
builder.add_node("token_counter", token_counter)
builder.set_finish_point("token_counter")